In [2]:
from keras.datasets import mnist
from keras.utils import np_utils
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
import os

# Training Parameters
batch_size = 64
epochs = 15

# loads the MNIST dataset
(x_train, y_train), (x_test, y_test)  = mnist.load_data()

# Lets store the number of rows and columns
img_rows = x_train[0].shape[0]
img_cols = x_train[1].shape[0]

# Getting our date in the right 'shape' needed for Keras
# We need to add a 4th dimenion to our date thereby changing our
# Our original image shape of (60000,28,28) to (60000,28,28,1)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

# store the shape of a single image 
input_shape = (img_rows, img_cols, 1)

# change our image type to float32 data type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize our data by changing the range from (0 to 255) to (0 to 1)
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Now we one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# Let's count the number columns in our hot encoded matrix 
print ("Number of Classes: " + str(y_test.shape[1]))

num_classes = y_test.shape[1]
num_pixels = x_train.shape[1] * x_train.shape[2]

# create model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss = 'categorical_crossentropy',
              optimizer = keras.optimizers.Adadelta(),
              metrics = ['accuracy'])

print(model.summary())

                     
checkpoint = ModelCheckpoint("/content/drive/My Drive/DeepLearningCV/12. Optimizers, Adaptive Learning Rate & Callbacks/MNIST_Checkpoint.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)
callbacks = [checkpoint]


Using TensorFlow backend.


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Number of Classes: 10





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
___________________________________

In [3]:
history = model.fit(x_train, y_train,
          batch_size = batch_size,
          epochs = epochs,
          verbose = 1,
          callbacks = callbacks,
          validation_data = (x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 60000 samples, validate on 10000 samples
Epoch 1/15





60000/60000 [==============================] - 16s 263us/step - loss: 0.2198 - acc: 0.9329 - val_loss: 0.0501 - val_acc: 0.9836

Epoch 00001: val_loss improved from inf to 0.05013, saving model to /content/drive/My Drive/DeepLearningCV/12. Optimizers, Adaptive Learning Rate & Callbacks/MNIST_Checkpoint.h5
Epoch 2/15
60000/60000 [==============================] - 13s 219us/step - loss: 0.0839 - acc: 0.9750 - val_loss: 0.0424 - val_acc: 0.9860

Epoch 00002: val_loss improved from 0.05013 to 0.04237, saving model to /content/drive/My Drive/DeepLearningCV/12. Optimizers, Adaptive Learning Rate & Callbacks/MNIST_Checkpoint.h5
Epoch 3/15
60000/60000 [==============================] - 13s 219us/step - loss: 0.0608 - acc: 0.9824 - val_loss: 0.0408 - val_acc: 0.9866

Epoch 00003: val_loss improved from 0.04237 to 0.04077, saving model

In [0]:
from keras.callbacks import EarlyStopping

earlystop = EarlyStopping(monitor = 'val_loss', # value being monitored for improvement
                          min_delta = 0, #Abs value and is the min change required before we stop
                          patience = 3, #Number of epochs we wait before stopping 
                          verbose = 1,
                          restore_best_weights = True) #keeps the best weigths once stopped

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

In [6]:
history = model.fit(x_train, y_train,
          batch_size=64,
          epochs=50,
          verbose=1,
          callbacks = callbacks,
          validation_data=(x_test, y_test))


score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 14s 225us/step - loss: 0.0269 - acc: 0.9920 - val_loss: 0.0269 - val_acc: 0.9920

Epoch 00001: val_loss did not improve from 0.02615
Epoch 2/50
60000/60000 [==============================] - 13s 222us/step - loss: 0.0282 - acc: 0.9916 - val_loss: 0.0299 - val_acc: 0.9909

Epoch 00002: val_loss did not improve from 0.02615
Epoch 3/50
60000/60000 [==============================] - 13s 221us/step - loss: 0.0263 - acc: 0.9920 - val_loss: 0.0291 - val_acc: 0.9912

Epoch 00003: val_loss did not improve from 0.02615
Epoch 4/50
60000/60000 [==============================] - 13s 221us/step - loss: 0.0267 - acc: 0.9925 - val_loss: 0.0325 - val_acc: 0.9908
Restoring model weights from the end of the best epoch

Epoch 00004: val_loss did not improve from 0.02615
Epoch 00004: early stopping
Test loss: 0.026911518605536002
Test accuracy: 0.992
